<a href="https://colab.research.google.com/github/ziviland/styleflow_ganspace_latent_editor/blob/master/Latent_vector_editor_using_StyleFlow_and_GANSpace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Manage dependecies and pull repository
!pip install tensorflow-gpu==1.15.0
!pip install torchdiffeq
!git clone https://github.com/ziviland/styleflow_ganspace_latent_editor.git

     |████████████████████████████████| 411.5MB 36kB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 512kB 41.9MB/s 
     |████████████████████████████████| 3.8MB 52.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=b75bb86cde3864e4f0d1c65e8f3b47b9be3346f441c9d9a78d16e4824c5a67ae
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [ ]:
# @title Init everything
from styleflow_ganspace_latent_editor import App, GANLatentFactory
gan_factory = GANLatentFactory()
app = App()
states = []
images = []

Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
Training T : 1.0
Number of trainable parameters of Point CNF: 1691649


In [ ]:
#@title Get latent from StyleGAN2
from PIL import Image
from IPython.display import display
from ipywidgets import Dropdown, interactive_output, fixed

def init_image(idx):
    initial_state = gan_factory.get_state(idx)
    global states 
    states = [initial_state] * 3
    initial_image = app.generate_image(initial_state)
    global images
    images = [initial_image] * 3
    display(Image.fromarray(initial_image).resize((400, 400)))

ui = Dropdown(options=range(1000), value=23)
out = interactive_output(init_image, {"idx" : ui})

display(ui, out)

Dropdown(index=23, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, …

Output()

In [ ]:
#@title Play around (wait a little bit after parameter change to see results)
from ipywidgets import IntSlider, FloatSlider, VBox, HBox, Label, Layout, Button, interactive_output
import copy
import matplotlib.pyplot as plt
import numpy as np

# helper method for 3 images (displays faster then HBox with Image widgets)
def get_figure(img_arr):
    plt.close('all')
    fig, axes = plt.subplots(1, len(img_arr), figsize=(22, 8))
    titles = ['initial', 'current', 'next']
    for i in range(len(img_arr)):
        axes[i].axis('off')
        axes[i].set_title(titles[i])
        axes[i].imshow(img_arr[i])
    return fig

layout_center = Layout(display="flex", justify_content="center", width="100%")

# flow controls
flow_attr_titles = ['gender', 'glasses', 'yaw', 'pitch', 'baldness', 'beard', 'age', 'expression']
flow_attr_min = [0, 0, -50, -30, 0, 0, 0, 0]
flow_attr_max = [1, 1, 50, 30, 1, 1, 60, 1]


flow_title = Label("Flow attributes", layout=layout_center)
flow_sliders = []
for i in range(len(flow_attr_titles)):
    if flow_attr_titles[i] in ['Yaw', 'Pitch', 'Age']:
        slider = IntSlider(
            value=states[0].flow_attributes[i],
            min=flow_attr_min[i],
            max=flow_attr_max[i],
            description=flow_attr_titles[i],
            continuous_update=False,
            orientation='horizontal',
            layout=layout_center
        )
    else:
        slider = FloatSlider(
            value=states[0].flow_attributes[i],
            min=flow_attr_min[i],
            max=flow_attr_max[i],
            step = 0.2,
            description=flow_attr_titles[i],
            continuous_update=False,
            orientation='horizontal',
            layout=layout_center
        )
    flow_sliders.append(slider)

flow_controls = VBox([flow_title] + flow_sliders, layout=Layout(display="flex", width="30%"))

# pca controls
# format component №, start layer, end layer
selected_components = {
    "pca_baldness" :      dict(comp=21, start=2, end=5, min=-5, max=10, scale=1),
    "hair_color" :        dict(comp=16, start=8, end=9, min=-10, max=10, scale=1),
    "eyes_size" :         dict(comp=33, start=6, end=8, min=-10, max=10, scale=-1),
    "eye_openness" :      dict(comp=54, start=7, end=8, min=-10, max=10, scale=-1),
    "eyebrow_thick" :     dict(comp=37, start=8, end=9, min=-10, max=10, scale=-1),
    "lipstick" :          dict(comp=34, start=10, end=11, min=-10, max=10, scale=1),
    "open_mouth" :        dict(comp=35, start=3, end=7, min=-3, max=3, scale=-1),
    "skin_tone" :         dict(comp=1, start=14, end=15, min=-10, max=10, scale=5)
}
scale = [1, 1, 1, ]

pca_title = Label("PCA attributes", layout=layout_center)
pca_sliders = [IntSlider(
    value=0,
    min=comp['min'],
    max=comp['max'],
    step=1,
    description=" ".join(name.split("_")),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    layout=layout_center
) for name, comp in selected_components.items()]

pca_controls = VBox([pca_title] + pca_sliders, layout=Layout(display="flex", width="30%"))

# draw images
def update_image(**attributes):
    current_state = states[1]
    next_state = current_state

    # apply flow transformations
    flow_attributes = list(attributes.values())[:8]
    for i in range(len(flow_attributes)):
        if flow_attributes[i] != current_state.flow_attributes[i]:
            next_state = app.change_flow_attribute(next_state, i, flow_attributes[i])

    # apply pca transformations
    pca_attributes = list(attributes.items())[8:]
    for i, (name, value) in enumerate(pca_attributes):
        
        if value != 0:
            next_state = app.change_pca_attribute(
                prev_image_state=next_state, 
                component_index=selected_components[name]['comp'],
                start_layer=selected_components[name]['start'],
                end_layer=selected_components[name]['end'],
                increment=(value * selected_components[name]['scale'])
            )

    states[2] = next_state
    next_image = app.generate_image(next_state)
    images[2] = next_image
    display(get_figure(images))

# save changes
save_button = Button(description='Save to current', button_style='info')
def save_changes(event):
    states[1] = states[2]
    images[1] = images[2]
    out.clear_output()
    with out:
        display(get_figure(images))
save_button.on_click(save_changes)

# altogether
ui = HBox([flow_controls, pca_controls, save_button])
out = interactive_output(update_image, {
    'gender':               flow_sliders[0],
    'glasses':              flow_sliders[1],
    'yaw':                  flow_sliders[2],
    'pitch':                flow_sliders[3],
    'flow_baldness':        flow_sliders[4],
    'beard':                flow_sliders[5],
    'age':                  flow_sliders[6],
    'expression' :          flow_sliders[7],

    'pca_baldness' :        pca_sliders[0],
    'hair_color' :          pca_sliders[1],
    'eyes_size' :           pca_sliders[2],
    'eye_openness' :        pca_sliders[3],
    'eyebrow_thick' :       pca_sliders[4],
    'lipstick' :            pca_sliders[5],
    'open_mouth' :          pca_sliders[6],
    'skin_tone' :           pca_sliders[7]
    }) 

display(ui, out)

Output()